# MIMIC Pre-Processing Notebook

This notebook runs through the pre-processing we perform to get a single table of patient data.

MIMIC-III exists in a multi table format however our work is focused on generating synthetic versions of single table data. As a result we pre-process the data by collecting files from randomised patients and bring them into a single file format.

This file contains numerous continuous, categorical and datetime columns. This range can then be used to fully test data synthesis methods and ensuring they are robust to a realistic healthcare EHR type dataset.

This notebook can create various size tables. These different sizes are shown in cell headers. We can create a small, medium, large set each with 11k, 81k and 217k rows respectively. Run the cells to get the dataset size that you desire. The difference between them is mainly that we allow multiple entries for specific patients depending on how many times they are admitted to a respective hospital.

NOTE: Currently our model does NOT support missing data and as a result date of death (from original MIMIC) is ignored - future work should look at handling this common trend in data.

In order to run this file you need:

- admissions.csv.gz
- CHARTEVENTS.csv.gz
- D_ITEMS.csv.gz
- ICUSTAYS.csv.gz
- OUTPUTEVENTS.csv.gz
- PATIENTS.csv.gz

These can all be downloaded if you have MIMIC-III access - be careful to ensure they are accessible to load depending on file structure

In [5]:
import pandas as pd
import numpy as np
import time
from random import randrange, seed
from tqdm import tqdm

## Data Imports

Load in the compressed csv files that MIMIC-III provides. We choose a selection from these files however this is arbitrary. You can create whatever table you wish using our formula just note that it must follow the data guidance to work with the current version of SynthVAE.

In [6]:
mimic_path = "../data/" # Path containing MIMIC data
output_path = "../data/" # Path to save the created datasets

admissions = pd.read_csv(mimic_path + "ADMISSIONS.csv")
chartevents = pd.read_csv(mimic_path + "CHARTEVENTS.csv", nrows=10000000)
icustays = pd.read_csv(mimic_path + "ICUSTAYS.csv")
items = pd.read_csv(mimic_path + "D_ITEMS.csv")
outputevents = pd.read_csv(mimic_path + "OUTPUTEVENTS.csv")
patients = pd.read_csv(mimic_path + "PATIENTS.csv")

## Augment Patient DOBs

Here we augment patients DOBs in the MIMIC-III set. We are basically just bringing the date of births forward to closer mimic a period in time closer to when SynthVAE was created.

In [7]:
seed(2021)
## Augment patient DOBs
new_dobs = []
dob_offset = []
for index, row in tqdm(patients.iterrows(), total=patients.shape[0]):
    years_diff_behind = len(
        pd.date_range(
            start=pd.to_datetime(row["DOB"], format="%Y-%m-%d %H:%M:%S"),
            end=pd.to_datetime("2021-12-01 00:00:00", format="%Y-%m-%d %H:%M:%S"),
            freq="Y",
        )
    )
    years_diff_ahead = len(
        pd.date_range(
            start=pd.to_datetime("2021-12-01 00:00:00", format="%Y-%m-%d %H:%M:%S"),
            end=pd.to_datetime(row["DOB"], format="%Y-%m-%d %H:%M:%S"),
            freq="Y",
        )
    )

    if (years_diff_behind != 0) and (
        pd.to_datetime(row["DOB"], format="%Y-%m-%d %H:%M:%S")
        < pd.to_datetime("1930-01-01 00:00:00", format="%Y-%m-%d %H:%M:%S")
    ):
        num_years = randrange(years_diff_behind - 80, years_diff_behind - 40)
        new_dobs.append(
            pd.to_datetime(row["DOB"], format="%Y-%m-%d %H:%M:%S")
            + pd.DateOffset(years=num_years)
        )
        dob_offset.append(num_years)
    elif (years_diff_ahead != 0) and (
        pd.to_datetime(row["DOB"], format="%Y-%m-%d %H:%M:%S")
        > pd.to_datetime("2021-12-01 00:00:00", format="%Y-%m-%d %H:%M:%S")
    ):
        num_years = randrange(years_diff_ahead + 30, years_diff_ahead + 50)
        new_dobs.append(
            pd.to_datetime(row["DOB"], format="%Y-%m-%d %H:%M:%S")
            - pd.DateOffset(years=num_years)
        )
        dob_offset.append(-num_years)
    else:
        new_dobs.append(pd.to_datetime(row["DOB"], format="%Y-%m-%d %H:%M:%S"))
        dob_offset.append(0)
patients["DOB"] = new_dobs
patients["DOB_offset"] = dob_offset



100%|██████████| 46520/46520 [00:31<00:00, 1493.63it/s]


## Produce small input (11k rows)

First we set the seed to get reproducible tables independent of user.

Secondly we merge all the respective tables we have chosen to form our single table as well as the columns that we desire.

Thirdly, we loop through the rows we are generating and add on the offset we added to the date of birth, in the cell above, to all the other relevant datetime columns.

Fourthly, we then ensure that the admit time is before the chart time and ensure that the datetime columns make sense in their given context.

Finally, we drop columns that are not required, reformat ages etc to make them appropriate all before saving the table.

In [8]:
seed(2021)
# Generate small input data file
mimic_table = admissions[['SUBJECT_ID','ETHNICITY','ADMITTIME','DISCHTIME','DISCHARGE_LOCATION']]
mimic_table = mimic_table.merge(patients[['SUBJECT_ID','GENDER','DOB']])
mimic_table = mimic_table.merge(icustays[['SUBJECT_ID','ICUSTAY_ID','FIRST_CAREUNIT']])
one_per_pat = chartevents.drop_duplicates(subset=['SUBJECT_ID','ICUSTAY_ID'])
mimic_table = mimic_table.merge(one_per_pat[['SUBJECT_ID','ICUSTAY_ID','CHARTTIME','ITEMID','VALUE','VALUEUOM']],on=['SUBJECT_ID','ICUSTAY_ID'])

new_admits = []
new_dischs = []
new_chart = []

for index, row in tqdm(mimic_table.iterrows(), total=mimic_table.shape[0]):
    admit_min = len(pd.date_range(start=pd.to_datetime(row['DOB'],format='%Y-%m-%d %H:%M:%S'),end=pd.to_datetime('2021-12-01 00:00:00',format='%Y-%m-%d %H:%M:%S'),freq='D'))
    stay_len = len(pd.date_range(start=pd.to_datetime(row['ADMITTIME'],format='%Y-%m-%d %H:%M:%S'),end=pd.to_datetime(row['DISCHTIME'],format='%Y-%m-%d %H:%M:%S'),freq='S'))

    num_days_admit = randrange(np.round(admit_min*0.25).astype(int),np.round(admit_min*0.9).astype(int)+5)
    num_days_disch = randrange(0,50)
    num_secs_chart = randrange(np.round(stay_len*0.01).astype(int)+1,np.round(stay_len*0.99).astype(int)+10)
    new_admit_date = pd.to_datetime(row['DOB'],format='%Y-%m-%d %H:%M:%S') + pd.DateOffset(days=num_days_admit)
    new_admits.append(new_admit_date)
    new_dischs.append(new_admit_date + pd.DateOffset(days=num_days_disch))
    new_chart.append(new_admit_date + pd.DateOffset(seconds=num_secs_chart))

mimic_table['ADMITTIME'] = new_admits
mimic_table['DISCHTIME'] = new_dischs
mimic_table['CHARTTIME'] = new_chart

mimic_table = mimic_table[(pd.to_datetime(mimic_table.ADMITTIME) < pd.to_datetime(mimic_table.CHARTTIME)) & (pd.to_datetime(mimic_table.DISCHTIME) > pd.to_datetime(mimic_table.CHARTTIME))]
mimic_table = mimic_table.merge(items[['ITEMID','LABEL']],on=['ITEMID'])
mimic_table.drop(['ICUSTAY_ID','ITEMID'],axis=1,inplace=True)
age_calc = pd.Timestamp('2021-12-01 00:00:00')
mimic_table['DOB'] = pd.to_datetime(mimic_table['DOB'], format='%Y-%m-%d %H:%M:%S')
mimic_table['age'] = (age_calc - mimic_table['DOB']).astype('<m8[Y]')
mimic_table = mimic_table.groupby('SUBJECT_ID').head(4)
mimic_table.to_csv(output_path + "MIMIC_{}_{}.csv".format(mimic_table.shape[0], time.strftime("%Y%m%d")), index=False)
print(f"Small input table saved, number of columns:  {mimic_table.shape[1]}, number of rows: {mimic_table.shape[0]}")

100%|██████████| 26927/26927 [00:31<00:00, 865.58it/s] 


Small input table saved, number of columns:  13, number of rows: 11040


In [9]:
mimic_table.head()


,SUBJECT_ID,ETHNICITY,ADMITTIME,DISCHTIME,DISCHARGE_LOCATION,GENDER,DOB,FIRST_CAREUNIT,CHARTTIME,VALUE,VALUEUOM,LABEL,age
0,23,WHITE,2004-07-07,2004-08-16,HOME HEALTH CARE,M,1974-07-17,SICU,2004-07-10 08:33:40,58.0,mmHg,Non Invasive Blood Pressure diastolic,47.0
1,23,WHITE,1998-10-18,1998-11-02,HOME HEALTH CARE,M,1974-07-17,SICU,1998-10-18 11:43:14,58.0,mmHg,Non Invasive Blood Pressure diastolic,47.0
2,34,WHITE,1997-04-12,1997-05-12,HOME,M,1962-07-18,CCU,1997-04-13 18:17:38,66.0,mmHg,Non Invasive Blood Pressure diastolic,59.0
3,34,WHITE,1980-03-31,1980-04-20,HOME HEALTH CARE,M,1962-07-18,CCU,1980-03-31 20:17:11,66.0,mmHg,Non Invasive Blood Pressure diastolic,59.0
4,107,HISPANIC OR LATINO,1997-12-30,1998-01-24,HOME,M,1975-04-02,MICU,1997-12-30 15:45:56,61.0,mmHg,Non Invasive Blood Pressure diastolic,46.0


## Produce medium input (81k rows)

The process to produce the table is exactly the same as the small table the only difference now is we add in an additional step inbetween the second and third steps.

This additional step chooses a data split where we are including multiple rows for that patient:

- Patients included in <b>split_one</b> are forced to only have one row in the dataset.

- Patients between <b>split_one</b> and <b>split_two</b> are forced to have only two rows in the dataset.

- Patients after <b>split_two</b> i.e. the remainder, are forced to have up to 100 rows in the dataset.

Note: Just because we specify <b>head(2)</b> or <b>head(100)</b> does NOT mean that every patient looped over will be given 2 or 100 rows in the single table. They will be given the amount of rows that they can feasibly have. I.e. if a patient in the middle set only has one row then <b>head(2)</b> is just going to put one row in the single table. Likewise <b>head(100)</b> for a patient with 5 rows, will only enter 5 rows into the single table. These values the rows a patient COULD have in the set provided they have that many rows available to us in MIMIC-III.

In [10]:
seed(2021)
# Generate mid-sized input file
mimic_table = admissions[['SUBJECT_ID','ETHNICITY','ADMITTIME','DISCHTIME','DISCHARGE_LOCATION']]
mimic_table = mimic_table.merge(patients[['SUBJECT_ID','GENDER','DOB']])
mimic_table = mimic_table.merge(icustays[['SUBJECT_ID','ICUSTAY_ID','FIRST_CAREUNIT']])

total_subjects = list(chartevents.SUBJECT_ID.unique())

df_list = []

split_one = int(np.round(len(total_subjects)*0.3).astype(int))
split_two = int(np.round(len(total_subjects)*0.9).astype(int))

for sub_index in range(0,split_one):
    search_id = total_subjects[sub_index]
    df_list.append(chartevents[chartevents.SUBJECT_ID == int(search_id)].head(1))

for sub_index in range(split_one,split_two):
    search_id = total_subjects[sub_index]
    df_list.append(chartevents[chartevents.SUBJECT_ID == int(search_id)].head(2))

for sub_index in range(split_two,len(total_subjects)):
    search_id = total_subjects[sub_index]
    df_list.append(chartevents[chartevents.SUBJECT_ID == int(search_id)].head(100))

one_per_pat = pd.concat(df_list)
one_per_pat.shape

mimic_table = mimic_table.merge(one_per_pat[['SUBJECT_ID','ICUSTAY_ID','CHARTTIME','ITEMID','VALUE','VALUEUOM']],on=['SUBJECT_ID','ICUSTAY_ID'])

new_admits = []
new_dischs = []
new_chart = []

patient_stays = mimic_table[['SUBJECT_ID','ICUSTAY_ID','DOB','ADMITTIME']]
patient_stays.drop_duplicates(inplace=True)

for index, row in tqdm(mimic_table.iterrows(), total=mimic_table.shape[0]):
    admit_min = len(pd.date_range(start=pd.to_datetime(row['DOB'],format='%Y-%m-%d %H:%M:%S'),end=pd.to_datetime('2021-12-01 00:00:00',format='%Y-%m-%d %H:%M:%S'),freq='D'))
    stay_len = len(pd.date_range(start=pd.to_datetime(row['ADMITTIME'],format='%Y-%m-%d %H:%M:%S'),end=pd.to_datetime(row['DISCHTIME'],format='%Y-%m-%d %H:%M:%S'),freq='S'))

    num_days_admit = randrange(np.round(admit_min*0.25).astype(int),np.round(admit_min*0.9).astype(int)+5)
    num_days_disch = randrange(0,50)
    num_secs_chart = randrange(np.round(stay_len*0.01).astype(int)+1,np.round(stay_len*0.99).astype(int)+10)
    new_admit_date = pd.to_datetime(row['DOB'],format='%Y-%m-%d %H:%M:%S') + pd.DateOffset(days=num_days_admit)
    new_admits.append(new_admit_date)
    new_dischs.append(new_admit_date + pd.DateOffset(days=num_days_disch))
    new_chart.append(new_admit_date + pd.DateOffset(seconds=num_secs_chart))

mimic_table['ADMITTIME'] = new_admits
mimic_table['DISCHTIME'] = new_dischs
mimic_table['CHARTTIME'] = new_chart

mimic_table = mimic_table[(pd.to_datetime(mimic_table.ADMITTIME) < pd.to_datetime(mimic_table.CHARTTIME)) & (pd.to_datetime(mimic_table.DISCHTIME) > pd.to_datetime(mimic_table.CHARTTIME))]
mimic_table = mimic_table.merge(items[['ITEMID','LABEL']],on=['ITEMID'])
mimic_table.drop(['ICUSTAY_ID','ITEMID'],axis=1,inplace=True)
age_calc = pd.Timestamp('2021-12-01 00:00:00')
mimic_table['DOB'] = pd.to_datetime(mimic_table['DOB'], format='%Y-%m-%d %H:%M:%S')
mimic_table['age'] = (age_calc - mimic_table['DOB']).astype('<m8[Y]')
mimic_table.to_csv(output_path + "MIMIC_{}_{}.csv".format(mimic_table.shape[0], time.strftime("%Y%m%d")), index=False)
print(f"Mid sized input table saved, number of columns:  {mimic_table.shape[1]}, number of rows: {mimic_table.shape[0]}")

/var/folders/fy/skyvcp4s3tj6z_xhj2sv7jf00000gn/T/ipykernel_42777/1106345372.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  patient_stays.drop_duplicates(inplace=True)
100%|██████████| 90891/90891 [01:14<00:00, 1222.72it/s]


Mid sized input table saved, number of columns:  13, number of rows: 81665


## Produce large input (217k rows)

Same rules apply in here as the medium set, only difference here is that <b>split_one</b> and <b>split_two</b> are different sizes allowing for more patients with up to 2 and up to 100 rows.

In [11]:
seed(2021)
# Generate large-sized input file
mimic_table = admissions[['SUBJECT_ID','ETHNICITY','ADMITTIME','DISCHTIME','DISCHARGE_LOCATION']]
mimic_table = mimic_table.merge(patients[['SUBJECT_ID','GENDER','DOB']])
mimic_table = mimic_table.merge(icustays[['SUBJECT_ID','ICUSTAY_ID','FIRST_CAREUNIT']])

total_subjects = list(chartevents.SUBJECT_ID.unique())

df_list = []

split_one = int(np.round(len(total_subjects)*0.3).astype(int))
split_two = int(np.round(len(total_subjects)*0.7).astype(int))

for sub_index in range(0,split_one):
    search_id = total_subjects[sub_index]
    df_list.append(chartevents[chartevents.SUBJECT_ID == int(search_id)].head(1))

for sub_index in range(split_one,split_two):
    search_id = total_subjects[sub_index]
    df_list.append(chartevents[chartevents.SUBJECT_ID == int(search_id)].head(2))

for sub_index in range(split_two,len(total_subjects)):
    search_id = total_subjects[sub_index]
    df_list.append(chartevents[chartevents.SUBJECT_ID == int(search_id)].head(100))

one_per_pat = pd.concat(df_list)
one_per_pat.shape

mimic_table = mimic_table.merge(one_per_pat[['SUBJECT_ID','ICUSTAY_ID','CHARTTIME','ITEMID','VALUE','VALUEUOM']],on=['SUBJECT_ID','ICUSTAY_ID'])

new_admits = []
new_dischs = []
new_chart = []

patient_stays = mimic_table[['SUBJECT_ID','ICUSTAY_ID','DOB','ADMITTIME']]
patient_stays.drop_duplicates(inplace=True)

for index, row in tqdm(mimic_table.iterrows(), total=mimic_table.shape[0]):
    admit_min = len(pd.date_range(start=pd.to_datetime(row['DOB'],format='%Y-%m-%d %H:%M:%S'),end=pd.to_datetime('2021-12-01 00:00:00',format='%Y-%m-%d %H:%M:%S'),freq='D'))
    stay_len = len(pd.date_range(start=pd.to_datetime(row['ADMITTIME'],format='%Y-%m-%d %H:%M:%S'),end=pd.to_datetime(row['DISCHTIME'],format='%Y-%m-%d %H:%M:%S'),freq='S'))

    num_days_admit = randrange(np.round(admit_min*0.25).astype(int),np.round(admit_min*0.9).astype(int)+5)
    num_days_disch = randrange(0,50)
    num_secs_chart = randrange(np.round(stay_len*0.01).astype(int)+1,np.round(stay_len*0.99).astype(int)+10)
    new_admit_date = pd.to_datetime(row['DOB'],format='%Y-%m-%d %H:%M:%S') + pd.DateOffset(days=num_days_admit)
    new_admits.append(new_admit_date)
    new_dischs.append(new_admit_date + pd.DateOffset(days=num_days_disch))
    new_chart.append(new_admit_date + pd.DateOffset(seconds=num_secs_chart))

mimic_table['ADMITTIME'] = new_admits
mimic_table['DISCHTIME'] = new_dischs
mimic_table['CHARTTIME'] = new_chart

mimic_table = mimic_table[(pd.to_datetime(mimic_table.ADMITTIME) < pd.to_datetime(mimic_table.CHARTTIME)) & (pd.to_datetime(mimic_table.DISCHTIME) > pd.to_datetime(mimic_table.CHARTTIME))]
mimic_table = mimic_table.merge(items[['ITEMID','LABEL']],on=['ITEMID'])
mimic_table.drop(['ICUSTAY_ID','ITEMID'],axis=1,inplace=True)
age_calc = pd.Timestamp('2021-12-01 00:00:00')
mimic_table['DOB'] = pd.to_datetime(mimic_table['DOB'], format='%Y-%m-%d %H:%M:%S')
mimic_table['age'] = (age_calc - mimic_table['DOB']).astype('<m8[Y]')
mimic_table.to_csv(output_path + "MIMIC_{}_{}.csv".format(mimic_table.shape[0], time.strftime("%Y%m%d")), index=False)
print(f"Large input table saved, number of columns:  {mimic_table.shape[1]}, number of rows: {mimic_table.shape[0]}")

/var/folders/fy/skyvcp4s3tj6z_xhj2sv7jf00000gn/T/ipykernel_42777/2970102133.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  patient_stays.drop_duplicates(inplace=True)
100%|██████████| 241438/241438 [02:52<00:00, 1398.56it/s]


Large input table saved, number of columns:  13, number of rows: 217068
